In [5]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download("stopwords")
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Reading the Dataset


In [15]:
df=pd.read_json("stackoverflow-data-idf.json",lines=True)
 
    
df

,id,title,body,answer_count,comment_count,creation_date,last_activity_date,last_editor_display_name,owner_display_name,owner_user_id,post_type_id,score,tags,view_count,accepted_answer_id,favorite_count,last_edit_date,last_editor_user_id,community_owned_date
0,4821394,Serializing a private struct - Can it be done?,<p>I have a public class that contains a priva...,1,0,2011-01-27 20:19:13.563 UTC,2011-01-27 20:21:37.59 UTC,,,163534.0,1,0,c#|serialization|xml-serialization,296,NaN,NaN,NaN,NaN,NaN
1,3367882,How do I prevent floated-right content from ov...,<p>I have the following HTML:</p>\n\n<pre><cod...,2,2,2010-07-30 00:01:50.9 UTC,2012-05-10 14:16:05.143 UTC,,,1190.0,1,2,css|overflow|css-float|crop,4121,3367943.0,0.0,2012-05-10 14:16:05.143 UTC,44390.0,NaN
2,31682135,Gradle command line,<p>I'm trying to run a shell script with gradl...,0,2,2015-07-28 16:30:18.28 UTC,2015-07-28 16:32:15.117 UTC,,,1299158.0,1,1,bash|shell|android-studio|gradle,259,NaN,NaN,NaN,NaN,NaN
3,20218536,Loop variable as parameter in asynchronous fun...,<p>I have an object with the following form.</...,1,1,2013-11-26 13:34:49.957 UTC,2013-11-26 15:07:50.8 UTC,,,642751.0,1,0,javascript|asynchronous|foreach|async.js,120,NaN,1.0,2013-11-26 15:02:47.993 UTC,1333873.0,NaN
4,19941459,Canot get the href value,<p>Hi I need to valid the href is empty or not...,5,1,2013-11-12 22:41:36.11 UTC,2013-11-12 23:48:34.67 UTC,,,819774.0,1,0,javascript,97,19941620.0,NaN,2013-11-12 22:43:42.97 UTC,21886.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,45643057,C# Telnet Error (Using CMD),<p>At first I want to say that my English is n...,0,1,2017-08-11 20:10:22.387 UTC,2017-08-11 22:00:08.853 UTC,,,8450318.0,1,0,c#|winforms|cmd|telnet,28,NaN,NaN,2017-08-11 22:00:08.853 UTC,5359302.0,NaN
19996,18079413,UI Control for presenting data base in the iph...,<p>I'm developing an application for CRM propo...,0,0,2013-08-06 11:51:37.463 UTC,2013-08-06 11:51:37.463 UTC,,,1939409.0,1,1,android|ios|ipad|xamarin.ios|xamarin.android,80,NaN,NaN,NaN,NaN,NaN
19997,39977022,How to make atom-typescript write config out o...,<p>I'm working in a team of 3 devs and we're w...,0,0,2016-10-11 12:14:43.26 UTC,2017-01-18 18:41:16.67 UTC,,,4102561.0,1,1,typescript|atom-editor,18,NaN,NaN,2017-01-18 18:41:16.67 UTC,2449905.0,NaN
19998,33328431,understanding for in angular loop in ng-repeat,<p>I was just going through this example onlin...,3,3,2015-10-25 10:03:00.587 UTC,2015-10-25 10:30:08.807 UTC,,,4195815.0,1,1,angularjs,164,NaN,NaN,NaN,NaN,NaN


# Preprocessing

In [16]:
import re
def prepareData( data ):
    data['Text'] = data['title'] + data['body']      # Extracting the title and the body only as those are what we only need
    corpus=[]
    for i in range (0,len(data)):
        doc=re.sub("</?.*?>"," <> ",data['Text'][i])  #Removing tags
        doc = re.sub('[^a-zA-Z]',' ',doc)         #Removing nonalphabetic charachters
        doc =re.sub(r"http\S+", "",doc)          #Removing Urls
        doc=doc.split()
        lemm=WordNetLemmatizer()        
        doc=[lemm.lemmatize(word) for word in doc]   # Doint lemmatization on the data
        doc= ' '.join(doc)
        corpus.append(doc)
    return corpus
    
corpus=prepareData(df)
    
 

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
stopWords=stopwords.words('english')

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopWords)
word_count_vector=cv.fit_transform(corpus)

In [43]:

list(cv.vocabulary_.keys())[:10]


['serializing',
 'private',
 'struct',
 'done',
 'public',
 'class',
 'contains',
 'property',
 'mostly',
 'string']

In [18]:
list(cv.get_feature_names())[2000:2015]


['adjustletterspacingtofitwidth',
 'adjustment',
 'adjustresize',
 'adjusts',
 'adjustsize',
 'adjusttokenprivileges',
 'adjusttypingstatus',
 'adjustviewbounds',
 'adk',
 'adksetup',
 'adl',
 'adlistener',
 'adlockpessimistic',
 'adlogo',
 'adm']

In [19]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)


TfidfTransformer()

In [20]:
df_test=pd.read_json("stackoverflow-test.json",lines=True)


# get test docs into a list
docs_test=prepareData(df_test)
docs_title=df_test['title'].tolist()
docs_body=df_test['body'].tolist()

In [21]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [22]:
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Title=====")
    print(docs_title[idx])
    print("\n=====Body=====")
    print(docs_body[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])


In [23]:
feature_names=cv.get_feature_names()
idx=5
keywords=get_keywords(idx)
print_results(idx,keywords)


=====Title=====
change CSS in external URL stylesheet - Wordpress

=====Body=====
<p>I am using a wordpress rating plugin that connects to an external URL stylesheet in order to display the ratings. </p>

<p>So I'd like to change the font-size of the stars that are displaying on my site.</p>

<p>I can see that the look of the font is defined in the following stylesheet and url: <code>https://publ.ratingz.com/app/FF897e.app.css</code> </p>

<p>The star font is using class <code>.rate-me-ratingz</code></p>

<p>I've been trying to change the font size using this class together with <code>!important</code> tag, but I guess that will not work, right?</p>

<p>Is there any way to edit and customise this external stylesheet in this case?</p>

===Keywords===
stylesheet 0.451
font 0.38
external 0.31
star 0.278
rating 0.273
wordpress 0.217
url 0.202
change 0.189
customise 0.184
size 0.144


In [38]:
idx=50
keywords=get_keywords(idx)
print_results(idx,keywords)


=====Title=====
How to display a viewController from an extended iOS module

=====Body=====
<p>I am creating an Module in Titanium which is extended in iOS. In that module I am creating a viewController and I want to show it on the screen when i called that module.
Not able to figure it out.. What will be right approach for it.</p>

===Keywords===
module 0.537
extended 0.437
viewcontroller 0.401
ios 0.312
creating 0.264
titanium 0.254
approach 0.148
screen 0.139
figure 0.137
called 0.123


In [30]:
tf_idf_vector=tfidf_transformer.transform(cv.transform(docs_test))

results=[]
for i in range(tf_idf_vector.shape[0]):
    
    # get vector for a single document
    curr_vector=tf_idf_vector[i]
    
    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    
    results.append(keywords)

df=pd.DataFrame(zip(docs,results),columns=['doc','keywords'])

In [31]:
df

,doc,keywords
0,serializing a private struct can it be done i ...,"{'eclipse': 0.49, 'maven': 0.451, 'war': 0.393..."
1,how do i prevent floated right content from ov...,"{'evaluate': 0.474, 'content': 0.403, 'console..."
2,gradle command line i m trying to run a shell ...,"{'appdomain': 0.41, 'dynamic': 0.384, 'perform..."
3,loop variable as parameter in asynchronous fun...,"{'background': 0.435, 'image': 0.405, 'jpg': 0..."
4,canot get the href value hi i need to valid th...,"{'uri': 0.371, 'bitmap': 0.319, 'intent': 0.30..."
...,...,...
495,how to unbind click and click submit button in...,"{'delphi': 0.699, 'compatible': 0.31, 'win': 0..."
496,swaggerui auth redirect swaggeruiauth of null ...,"{'node': 0.508, 'null': 0.324, 'selectsingleno..."
497,ssrs value display error for ssrs conditional ...,"{'logo': 0.529, 'step': 0.318, 'triangle': 0.3..."
498,accessing and changing a class instance from a...,"{'length': 0.402, 'ev': 0.391, 'introduce': 0...."


In [2]:
from keybert import KeyBERT


kw_model = KeyBERT()

In [29]:
kw_model.extract_keywords(docs_test[5], keyphrase_ngram_range=(1, 1), stop_words='english',
                              use_maxsum=True, nr_candidates=20, top_n=10)

[('way', 0.0937),
 ('important', 0.1023),
 ('url', 0.1486),
 ('publ', 0.1513),
 ('star', 0.1738),
 ('external', 0.1816),
 ('rate', 0.2769),
 ('customise', 0.3246),
 ('font', 0.3279),
 ('stylesheet', 0.4175)]

In [28]:
docs_test[5]

'change CSS in external URL stylesheet Wordpress I am using a wordpress rating plugin that connects to an external URL stylesheet in order to display the rating So I d like to change the font size of the star that are displaying on my site I can see that the look of the font is defined in the following stylesheet and url publ ratingz com app FF e app cs The star font is using class rate me ratingz I ve been trying to change the font size using this class together with important tag but I guess that will not work right Is there any way to edit and customise this external stylesheet in this case'